<a href="https://colab.research.google.com/github/hejazn86/dogs_vs_cats_classifier/blob/master/notebook_with_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/drive')


In [ ]:
from tensorflow.keras import layers , models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

 
pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = 'imagenet')


for layer in pre_trained_model.layers:
  layer.trainable = False

#pre_trained_model.summary()
last_layer = pre_trained_model.get_layer('mixed10')
print(last_layer.output_shape) 
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

classifier = models.Model(pre_trained_model.input, x)

classifier.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])

In [ ]:
import os
import numpy as np

directory = '/drive/My Drive/python/cats_and_dogs_filtered'
train_dir = os.path.join(directory, 'train')
val_dir = os.path.join(directory, 'validation')
train_dogs_dir = os.path.join(train_dir, 'dogs')
train_cats_dir = os.path.join(train_dir, 'cats')
test_dogs_dir = os.path.join(val_dir, 'dogs')
test_cats_dir = os.path.join(val_dir, 'cats')

train_dog_files = os.listdir(train_dogs_dir)
train_cat_files = os.listdir(train_cats_dir)
test_dog_files = os.listdir(test_dogs_dir)
test_cat_files = os.listdir(test_cats_dir)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy  as np

train_data_generator = ImageDataGenerator(rescale= 1.0/255.0,
                                          rotation_range = 40,
                                          width_shift_range = 0.2,
                                          height_shift_range = 0.2,
                                          shear_range = 0.2,
                                          zoom_range = 0.2,
                                          horizontal_flip = True,
                                          fill_mode = 'nearest')

val_data_generator = ImageDataGenerator(rescale= 1.0/ 255.0)


train_generator = train_data_generator.flow_from_directory(train_dir,
                                                           target_size = (150, 150),
                                                           batch_size = 40,
                                                           class_mode = 'binary')
val_generator = val_data_generator.flow_from_directory(val_dir,
                                                       target_size = (150, 150),
                                                       batch_size = 20,
                                                       class_mode = 'binary')

print(train_generator.dtype)


In [ ]:
history = classifier.fit_generator(train_generator, steps_per_epoch = 50, epochs = 20, validation_data = val_generator, validation_steps = 50)

classifier.summary()



In [ ]:

predicted = classifier.predict(val_generator)
val_loss, val_acc = classifier.evaluate(val_generator, steps = 5)


import matplotlib.pyplot as plt
import numpy as np

epochs = range(len(history.history['acc']))

plt.figure(figsize = (12, 7))
plt.subplot(121).plot(epochs, history.history['loss'], 'b*', label = 'Train_loss')
plt.subplot(121).plot(epochs, history.history['val_loss'], 'r', label = 'Test_loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.subplot(122).plot(epochs, history.history['acc'], 'b*', label = 'Train_accuracy')
plt.subplot(122).plot(epochs, history.history['val_acc'], 'r', label = 'Test_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.grid(True)
plt.legend(['Train', 'Test'], loc = 'uper right')
plt.show()